In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

Reviews and respective sentiments

In [2]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

After one hot encoding the words

In [3]:
#30 represents the vocab size you want
one_hot("amazing restaurant",30)

[6, 7]

Encoding the words in the dataset

In [4]:
vocab_size = 30
encoded_reviews = [one_hot(d, vocab_size) for d in reviews]
print(encoded_reviews)

[[23, 16], [6, 7], [23, 15], [21, 23, 7], [3, 15, 10], [6, 16], [27, 15, 5], [8, 8], [8, 24], [19, 10]]


In [6]:
max_length = 3 #represents max length of review,after which a padding word would occur
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print(padded_reviews)

[[23 16  0]
 [ 6  7  0]
 [23 15  0]
 [21 23  7]
 [ 3 15 10]
 [ 6 16  0]
 [27 15  5]
 [ 8  8  0]
 [ 8 24  0]
 [19 10  0]]


In [7]:
embeded_vector_size = 5 #represents the number of features for reviews/size of embedded vector you want

Model

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, embeded_vector_size, input_length=max_length,name="embedding"))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid')) #output neuron is 1 because its a binary classification

In [9]:
X = padded_reviews
y = sentiment

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 5)              150       
                                                                 
 flatten (Flatten)           (None, 15)                0         
                                                                 
 dense (Dense)               (None, 1)                 16        
                                                                 
Total params: 166
Trainable params: 166
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model.fit(X, y, epochs=50, verbose=0)

In [12]:
# evaluate the model
loss, accuracy = model.evaluate(X, y)
accuracy

1/1 [==============================] - 0s 185ms/step - loss: 0.6225 - accuracy: 1.0000


1.0

Explore the weights in the embedding layer

In [13]:
weights = model.get_layer('embedding').get_weights()[0] #you can load these embedding for another task,completely different from this too
len(weights) ##padded review length x number of reviews

30

In [14]:
weights[13]

array([ 0.02005008,  0.00483872, -0.02585535,  0.04874237,  0.03584326],
      dtype=float32)